# A. Récupération des données  

"description"

## A.1 - Choix des données 
"explication"
- choix de la base 
- choix du cours 
- choix des dates (strat + end)

## A.2 - Importation et traitement des données

requierments : "pip install yfinance"

In [2]:
import yfinance as yf

ticker = "NVDA"
start_date = "2015-01-01"
end_date = "2024-12-31"

# TODO : edit le text en anglais vers FR   
# Download historical stock data
# We use the yfinance  API to scrape open market data from Yahoo Finance.
# This satisfies the requirement of using open data sources.
nvda_data = yf.download(ticker, start=start_date, end=end_date)

# Save original raw data as per requirements
nvda_data.to_csv("nvda_stock_data_raw.csv")
print("Raw dataset saved to nvda_stock_data_raw.csv")


nvda_data.head()
# Reset index to make Date a column
nvda_data.reset_index(inplace=True)

# Flatten MultiIndex columns created by yfinance
nvda_data.columns = ['_'.join(col).strip() if isinstance(col, tuple) and col[1] else col[0] for col in nvda_data.columns.values]
nvda_data.head()

# Sort data by date
nvda_data = nvda_data.sort_values(by='Date')
nvda_data.head()

ModuleNotFoundError: No module named 'yfinance'